In [1]:
import os
import warnings

import dask
import ncar_jobqueue
import yaml

import utils

%matplotlib inline

In [2]:
with open("diag_metadata.yaml", mode="r") as fptr:
    diag_metadata_list = yaml.safe_load(fptr)

# varnames = utils.get_varnames_from_metadata_list(diag_metadata_list)

In [3]:
def summary_plots(ds, diag_metadata, save_pngs=False):
    varname = diag_metadata["varname"]
    print(varname)
    da = ds[varname].isel(diag_metadata.get("isel_dict"))

    utils.summary_plot_global_ts(
        ds,
        da,
        diag_metadata,
        time_coarsen_len=12,
        save_pngs=save_pngs,
        casename=ds.attrs["title"],
    )

    utils.summary_plot_histogram(
        ds, da, diag_metadata, save_pngs=save_pngs, casename=ds.attrs["title"]
    )

In [4]:
cluster = ncar_jobqueue.NCARCluster(
    cores=2, memory="64 GB", processes=2, walltime="6:00:00"
)
cluster.scale(n=8)  # n = number of workers
print(f"dashboard_link={cluster.dashboard_link}")

dashboard_link=https://jupyterhub.ucar.edu/dav/user/mlevy/proxy/41844/status


/glade/work/mlevy/miniconda3/envs/hires-marbl/lib/python3.7/site-packages/distributed/dashboard/core.py:79: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [5]:
casename = "g.e22b05.G1850ECOIAF_JRA.TL319_g17.cocco.001"
year = 95
campaign_root = os.path.join(
    os.path.sep,
    "glade",
    "campaign",
    "cesm",
    "development",
    "bgcwg",
    "projects",
    "1deg_cocco_JRA",
    "cases",
)

# Set up CaseClass object
case = utils.CaseClass(casename, os.path.join(campaign_root, casename))

In [6]:
with dask.distributed.Client(cluster) as client:
    stream = "pop.h"
    #     ds = case.gen_dataset(varnames, stream)
    for diag_metadata in diag_metadata_list:
        ds = case.gen_dataset(
            diag_metadata["varname"], stream, start_year=95, end_year=104
        )
        summary_plots(ds, diag_metadata, save_pngs=True)

Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
POC_FLUX_100m
Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
CaCO3_FLUX_100m
Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
FG_CO2
Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
DpCO2
Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
PH
Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
spChl
Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
photoC_sp_zint
Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
coccoChl
Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
photoC_cocco_zint
Datasets contain a total of 120 time samples
Last average written at 0105-01-01 00:00:00
diatChl
Datasets

In [7]:
cluster.close()